This notebook will be sketching out my implementation of the Resnet-18 network used in the [paper](http://cs231n.stanford.edu/reports/2017/pdfs/406.pdf) mentioned in initial_work.ipynb. I want to try building it out with the base components built into PyTorch and compare that to PyTorch's built-in Resnet constructors.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import common_utils

class HomegrownResnet18(nn.Module):
    
    def __init__(self):
        super(HomegrownResnet18, self).__init__()
        # 3 input channels, 64 output channels, 3x3 square convolution kernel
        self.in = nn.Conv2d(3,64,3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(64,32,3, stride=2, padding=1)
        self.fc1 = nn.Linear(6272,228)
        self.fc2 = nn.Linear(228,common_utils.get_num_artists())
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 6272)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        
net = HomegrownResnet18()
    
print(net)

HomegrownResnet18(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (fc1): Linear(in_features=6272, out_features=228, bias=True)
  (fc2): Linear(in_features=228, out_features=38, bias=True)
)


In [2]:
train, test, val = common_utils.get_dataloaders()

In [ ]:
import torch.optim as optim

#make sure we do things on the gpu
net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

import matplotlib.pyplot as plt
#re init
train_loader = torch.utils.data.DataLoader(artist_image_dataset,
                                           batch_size=64,
                                           num_workers=2,
                                           sampler=train_sampler)

test_acc = []
iterations = []
train_len = len(train_loader)

for epoch in range(10):
    
    running_loss = 0
    
    for i, sample in enumerate(train_loader):
        images, labels = sample['images'].cuda(), sample['labels'].cuda()
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 20 == 19:
            print(i+1)
        if i % 50 == 49:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
            with torch.no_grad():
                total = 0
                correct = 0
                for sample in test_loader:
                    images,labels = sample['images'].cuda(), sample['labels'].cuda()
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                test_acc.append(100*correct/total)
                iterations.append((i+1) + (train_len * epoch))
                print("[%d, %5d] test accuracy: %.3f" % (epoch + 1, i + 1, 100*correct/total))
                    
print('done')


In [ ]:
plt.plot(iterations, test_acc)
plt.xlabel('number of iterations')
plt.ylabel('percent accuracy')
plt.show()